In [1]:
import rclpy


In [1]:
import sys
print(sys.executable)


/usr/bin/python3


In [2]:
import os

print("Exists color image:", os.path.isfile("1_rgb_Color.png"))
print("Exists depth image:", os.path.isfile("1_Depth.png"))


Exists color image: False
Exists depth image: True


In [ ]:
import open3d as o3d
import numpy as np
import os
import cv2

image_dir = "snaps"
num_frames = 11

# Depth image resolution
depth_width, depth_height = 424, 240
fx, fy = 600, 600
cx, cy = depth_width / 2, depth_height / 2

intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=depth_width, height=depth_height, fx=fx, fy=fy, cx=cx, cy=cy)

pcd_combined = o3d.geometry.PointCloud()

for i in range(1, num_frames + 1):
    color_path = os.path.join(image_dir, f"{i}_Color.png")
    depth_path = os.path.join(image_dir, f"{i}_Depth.png")
    
    if not os.path.exists(color_path):
        print(f"❌ Color image not found: {color_path}")
        continue
    if not os.path.exists(depth_path):
        print(f"❌ Depth image not found: {depth_path}")
        continue

    # Load color image with OpenCV and resize
    color_cv = cv2.imread(color_path)
    if color_cv is None:
        print(f"⚠️ Skipping frame {i}: Failed to load color image")
        continue
    color_cv_resized = cv2.resize(color_cv, (depth_width, depth_height))  # Match depth resolution
    color_raw = o3d.geometry.Image(cv2.cvtColor(color_cv_resized, cv2.COLOR_BGR2RGB))

    # Load depth image using Open3D
    depth_raw = o3d.io.read_image(depth_path)

    print(f"✅ Frame {i}: Loaded and resized")

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw,
        depth_raw,
        depth_scale=1000.0,
        depth_trunc=3.0,
        convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)

    # Flip the point cloud for correct orientation
    pcd.transform([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])

    pcd_combined += pcd

# Visualize the final point cloud
o3d.visualization.draw_geometries([pcd_combined])


In [ ]:
import open3d as o3d
import numpy as np
import cv2
import os
import struct

image_dir = "snaps"
num_frames = 11

# Depth resolution
depth_width, depth_height = 424, 240
fx, fy = 600.0, 600.0
cx, cy = depth_width / 2.0, depth_height / 2.0

intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(depth_width, depth_height, fx, fy, cx, cy)

pcd_combined = o3d.geometry.PointCloud()

for i in range(1, num_frames + 1):
    color_path = os.path.join(image_dir, f"{i}_Color.png")
    depth_raw_path = os.path.join(image_dir, f"{i}_Depth.raw")
    
    if not os.path.exists(color_path) or not os.path.exists(depth_raw_path):
        print(f"❌ Missing files for frame {i}")
        continue

    # Load color image
    color_cv = cv2.imread(color_path)
    color_resized = cv2.resize(color_cv, (depth_width, depth_height))
    color_o3d = o3d.geometry.Image(cv2.cvtColor(color_resized, cv2.COLOR_BGR2RGB))

    # Load raw depth data (assumed 16-bit unsigned short in .raw file)
    with open(depth_raw_path, "rb") as f:
        depth_data = np.frombuffer(f.read(), dtype=np.uint16).reshape((depth_height, depth_width))

    # Optional: Bilateral filtering to clean up noise (optional and slow)
    # depth_data = cv2.bilateralFilter(depth_data.astype(np.float32), d=5, sigmaColor=75, sigmaSpace=75).astype(np.uint16)

    depth_o3d = o3d.geometry.Image(depth_data)

    print(f"✅ Frame {i} loaded: color and depth")

    # Combine into RGBD image
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_o3d,
        depth_o3d,
        depth_scale=1000.0,
        depth_trunc=3.0,
        convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)

    # Flip to match Open3D coordinate system
    pcd.transform([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])

    pcd_combined += pcd

# Optional: downsample point cloud
# pcd_combined = pcd_combined.voxel_down_sample(voxel_size=0.005)

# Visualize
o3d.visualization.draw_geometries([pcd_combined])

# Save result
# o3d.io.write_point_cloud("chair_pointcloud.ply", pcd_combined)
o3d.io.write_point_cloud("combined_pointcloud.ply", pcd_combined)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
import open3d as o3d
import numpy as np

# Load combined point cloud (from previous step)
pcd = pcd_combined

# Estimate normals for plane segmentation
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=0.02, max_nn=30))

# Segment the largest plane using RANSAC
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"✅ Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# Extract points that are NOT part of the plane
pcd_without_plane = pcd.select_by_index(inliers, invert=True)

# Optional: Remove outliers
pcd_clean = pcd_without_plane.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)[0]

# Optional: Further cluster to isolate the largest object (likely the chair)
labels = np.array(pcd_clean.cluster_dbscan(eps=0.02, min_points=20, print_progress=True))
largest_cluster_idx = np.argmax(np.bincount(labels[labels >= 0]))
chair = pcd_clean.select_by_index(np.where(labels == largest_cluster_idx)[0])

# Visualize
o3d.visualization.draw_geometries([chair], window_name="Chair Isolated")

# Save result
o3d.io.write_point_cloud("/home/chair_only.ply", chair)



In [ ]:
# Poisson surface reconstruction (best for closed surfaces)
mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(chair, depth=9)

# Crop to bounding box (optional, removes weird outer mesh)
bbox = chair.get_axis_aligned_bounding_box()
mesh = mesh.crop(bbox)

# Save the mesh
o3d.io.write_triangle_mesh("chair_mesh.dae", mesh)  # or .stl


In [1]:
import open3d as o3d

# Load PLY file
pcd = o3d.io.read_point_cloud("vedh_ros/combined_pointcloud.ply")

# Estimate mesh from point cloud (Poisson surface reconstruction, optional)
pcd.estimate_normals()
mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# Save to DAE
o3d.io.write_triangle_mesh("your_model.dae", mesh)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Read PLY failed: unable to read file: vedh_ros/combined_pointcloud.ply


RPly: Error reading 'y' of 'vertex' number 38828


[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.


[WARNING] /root/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.Initialize.inl (Line 218)
          Initialize
          Found bad sample nodes: 1


False

In [6]:
import os
print("📁 Saved to:", os.path.abspath("chair_only.ply"))


📁 Saved to: /home/vedh/chair_only.ply
